## Блок 03 - Первые отчеты по API

In [121]:
import requests
import yaml
from pprint import pprint 

f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.net/stat/v1/data'
startDate = '2022-01-01'
endDate = '2022-01-07'
counter = '53600281' #'21075004'
dimensions = ['ym:s:date']
metrics = ['ym:s:visits', 'ym:s:users', 'ym:s:pageviews', 'ym:s:percentNewVisitors'] #, 'ym:s:newUsers']
dimensions_string = ','.join(dimensions)
metrics_string = ','.join(metrics)
sort = dimensions
params = {
    'ids': counter,
    'date1': startDate,
    'date2': endDate,
    'group': 'day',
    'sort': sort,
    'dimensions': dimensions_string,
    'metrics': metrics_string,
    'oauth_token': token
}

In [122]:
r = requests.get(API_URL, params= params)
data = r.json()

In [123]:
pprint(data)

{'contains_sensitive_data': False,
 'data': [{'dimensions': [{'name': '2022-01-01'}],
           'metrics': [541.0, 502.0, 732.0, 88.24701195]},
          {'dimensions': [{'name': '2022-01-02'}],
           'metrics': [686.0, 647.0, 864.0, 86.24420402]},
          {'dimensions': [{'name': '2022-01-03'}],
           'metrics': [827.0, 772.0, 1041.0, 90.67357513]},
          {'dimensions': [{'name': '2022-01-04'}],
           'metrics': [933.0, 880.0, 1171.0, 88.06818182]},
          {'dimensions': [{'name': '2022-01-05'}],
           'metrics': [943.0, 895.0, 1200.0, 88.15642458]},
          {'dimensions': [{'name': '2022-01-06'}],
           'metrics': [998.0, 935.0, 1232.0, 87.70053476]},
          {'dimensions': [{'name': '2022-01-07'}],
           'metrics': [800.0, 770.0, 1034.0, 86.49350649]}],
 'data_lag': 187,
 'max': [998.0, 935.0, 1232.0, 90.67357513],
 'min': [541.0, 502.0, 732.0, 86.24420402],
 'query': {'adfox_event_id': '0',
           'attr_name': '',
           'attribut

In [124]:
for line in data['data']:
    print(line)

{'dimensions': [{'name': '2022-01-01'}], 'metrics': [541.0, 502.0, 732.0, 88.24701195]}
{'dimensions': [{'name': '2022-01-02'}], 'metrics': [686.0, 647.0, 864.0, 86.24420402]}
{'dimensions': [{'name': '2022-01-03'}], 'metrics': [827.0, 772.0, 1041.0, 90.67357513]}
{'dimensions': [{'name': '2022-01-04'}], 'metrics': [933.0, 880.0, 1171.0, 88.06818182]}
{'dimensions': [{'name': '2022-01-05'}], 'metrics': [943.0, 895.0, 1200.0, 88.15642458]}
{'dimensions': [{'name': '2022-01-06'}], 'metrics': [998.0, 935.0, 1232.0, 87.70053476]}
{'dimensions': [{'name': '2022-01-07'}], 'metrics': [800.0, 770.0, 1034.0, 86.49350649]}


In [125]:
# Считаем среднее значение доли новых посетителей по дням
new_visitors = 0
visitors = 0
for line in data['data']:
    visits_date = line['dimensions'][0]['name']
    visits = line['metrics'][0]
    users = line['metrics'][1]
    pageviews = line['metrics'][2]
    new_visitors_in_percents = line['metrics'][3]
    print(visits_date, int(visits), int(users), int(pageviews), round(new_visitors_in_percents, 2))
    new_visitors += round((users * new_visitors_in_percents) / 100)
    visitors += users
print(new_visitors, visitors, round(new_visitors / visitors, 4))

2022-01-01 541 502 732 88.25
2022-01-02 686 647 864 86.24
2022-01-03 827 772 1041 90.67
2022-01-04 933 880 1171 88.07
2022-01-05 943 895 1200 88.16
2022-01-06 998 935 1232 87.7
2022-01-07 800 770 1034 86.49
4751 5401.0 0.8797


In [126]:
# Доля новых пользователей в веб-интерфейсе за всю неделю оказалась заметно выше 
# (значение 92.04% в строке "Итого и средние"), чем за каждый день в отдельности 
# (за 07.01 - 86,49%, за 06.01 - 87,7% итд). Какой результат верен - итоговый или по дням? 
# Ведь не может среднее за неделю быть больше, чем значение за каждый день в отдельности.

# Ответ
# Оба показателя верны, т.к. в недельной статистике "Ежедневные новые пользователи" в течении недели будут
# учитываться только один раз.

In [131]:
# Шаблоны отчётов
import requests
import yaml

f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.net/stat/v1/data'
startDate = '2022-01-01'
endDate = '2022-01-07'
counter = '53600281' #'21075004'
sort = dimensions
preset = 'traffic'
params = {
    'ids': counter,
    'date1': startDate,
    'date2': endDate,
    'group': 'day',
    #'sort': sort,
    'preset': preset,
    'oauth_token': token
}
r = requests.get(API_URL, params= params)
data = r.json()

In [135]:
# Количество метрик в пресете
len(data['query']['metrics'])

7

In [164]:
# Фильтры в отчётах
import requests
import yaml

f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.net/stat/v1/data'
startDate = '2022-01-01'
endDate = '2022-01-07'
counter = '44147844'
sort = dimensions
preset = 'traffic'

filters = "ym:s:regionCountryName=='Russia'"

params = {
    'ids': counter,
    'date1': startDate,
    'date2': endDate,
    #'group': 'day',
    #'sort': sort,
    'preset': preset,
    'filters': filters,
    'oauth_token': token
}
r = requests.get(API_URL, params= params)
data = r.json()

In [165]:
pprint(data['data'])

[{'dimensions': [{'from': '2022-01-03',
                  'id': '2022-01-03',
                  'name': '2022-01-03 - 2022-01-07',
                  'to': '2022-01-07'}],
  'metrics': [283.0,
              261.0,
              441.0,
              89.65517241,
              35.33568905,
              1.55830389,
              76.54416961]},
 {'dimensions': [{'from': '2022-01-01',
                  'id': '2021-12-27',
                  'name': '2022-01-01 - 2022-01-02',
                  'to': '2022-01-02'}],
  'metrics': [125.0, 124.0, 180.0, 92.74193548, 43.2, 1.44, 51.68]}]


In [170]:
# visits_from_Russia - число визитов из России
visits_from_Russia = data['data'][0]['metrics'][0]
print(visits_from_Russia)

283.0


In [172]:
# Получим общее число посещений, для этого отключим фильтр
params = {
    'ids': counter,
    'date1': startDate,
    'date2': endDate,
    #'group': 'day',
    #'sort': sort,
    'preset': preset,
    #'filters': filters,
    'oauth_token': token
}
r = requests.get(API_URL, params= params)
data = r.json()

In [174]:
visits = data['data'][0]['metrics'][0]
print(visits)

1745.0


In [175]:
# Доля визитов из России
print(round(visits_from_Russia/visits, 2))

0.16
